# AM Microstucture

In [ ]:
import pyvista as pv

pv.set_jupyter_backend("static")

%load_ext autoreload
%autoreload 2

In [ ]:
from materialite.models.convolution_microstructure_model import (
    ConvolutionMicrostructureGPUModel,
    LaserBeam,
)
from materialite import get_ipf_colors
from materialite.tensor import Orientation
from materialite import Material
import numpy as np
import pandas as pd

Materialite implements a method similar to that of (Rodgers et al. 2021) alongside the convolution method for simulating the microstructure field resulting from laser heating. This model currently requires a GPU and CuPy/CUDA and will raise an error if a compatible device is not available.

First, a material has to be created. Next, orientations and a related "spin" field (which typically represents grain ID values) need to be formed. The orientation reflects the unique crystallographic texture associated with each spin value.  

In [ ]:
material = Material(dimensions=[400, 50, 40], spacing=[5e-6, 5e-6, 5e-6])
material = material.create_uniform_field("temperature", 300.0)

num_spins = 10000
orientations = Orientation.random(num_spins)
orientations[0] = Orientation.from_euler_angles([0, 0, 0])
unique_spins = np.arange(num_spins)
region_field = pd.DataFrame({"spin": unique_spins, "orientation": orientations})
material = material.create_random_integer_field(
    "spin", low=1, high=num_spins
).create_regional_fields(region_label="spin", regional_fields=region_field)

Additionally, a laser path needs to be created:

In [ ]:
laser = LaserBeam(
    material,
    laser_power=200,
    laser_velocity=1.0,
    hatch_spacing=20,
    beam_x_radius=50.0e-6,
    beam_y_radius=50.0e-6,
    beam_z_radius=30.0e-6,
    num_layers=1,
    scan_offset_start=-10,
    hatch_offset_start=25,
    hatch_offset_end=-25,
    z_start=40,
    layer_thickness=8,
    time_between_scans=0.001,
    time_between_layers=0.0,
    rotation_angle_increment=np.deg2rad(67),
)

laser.create_build_path(material, adjust_domain=True, domain_epsilon=5e-6)
laser.plot_beam_path()

The microstructure model takes in similar parameters as the temperature field models:

In [ ]:
density = 4200.0
specific_heat_capacity = 656.2
thermal_conductivity = 18.72
volumetric_specific_heat = specific_heat_capacity * density
thermal_diffusivity = thermal_conductivity / volumetric_specific_heat

thermophysical_props = {
    "thermal_diffusivity": thermal_diffusivity,
    "volumetric_specific_heat": volumetric_specific_heat,
    "power_absorptivity": 0.48,
    "melt_temperature": 1928.0,
}

enthalpy_props = {
    "vapor_temperature": 3000.0,
    "heat_of_fusion": 370000.0 * density,
    "heat_of_vapor": 10000000 * density,
    "melt_range": 25.0,
    "vapor_range": 10.0,
}

model = ConvolutionMicrostructureGPUModel(
    save_frequency=50,
    time_steps=1000,
    thermophysical_props=thermophysical_props,
    enthalpy_props=enthalpy_props,
    adjust_xy_radii_multiple=3,
    adjust_xy_extra_voxels=0,
    z_range=60,
    orientations=orientations,
)

In [ ]:
new_material = model.run(material, laser, "temperature", "spin")

In [ ]:
new_material.plot("spin")

The IPF map can be plotted as well:

In [ ]:
ipf_colors = get_ipf_colors(
    specimen_frame_direction=[0, 0, 1], orientations=orientations, unit_cell="fcc"
)
ipf_colors[0] = [0, 0, 0]
ipf_region_field = pd.DataFrame(
    {"spin": unique_spins, "ipf_color": list(ipf_colors)}
)  # using the same unique_spins from before
new_material.create_regional_fields("spin", ipf_region_field).plot(
    "ipf_color", kind="ipf_map"
)